In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.datasets.reuters import load_data, get_word_index       # Reuters news data.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
warnings.filterwarnings('ignore')                  # Turn the warnings off.
%matplotlib inline

1.1 Read in the data:

In [ ]:
n_words = 1000                                        # Size of the vocabulary.
(X_train, y_train), (X_test, y_test) = load_data(num_words = n_words, test_split = 0.3)
n_train_size = X_train.shape[0]

In [ ]:
# Check for the shapes.
print("-"*50)
print("Training data X shape: {}".format(X_train.shape))
print("Training data y shape: {}".format(y_train.shape))
print("-"*50)
print("Test data X shape: {}".format(X_test.shape))
print("Test data y shape: {}".format(y_test.shape))
print("-"*50)

1.2 Explore the data

In [ ]:
# Number of unique values of y = Number of categories of the newswires.
n_cat = pd.Series(y_train).nunique()
n_cat

In [ ]:
# Print out an observation (document) contained in X
# It is encoded as integers (indices)
print(X_train[0])

In [ ]:
# Let's check for length of the first 100 documents.
# We notice that the length is not uniform
print([len(a) for a in X_train[0:100]])

In [ ]:
# Download the dictionary to translate the indices
my_dict = get_word_index(path = 'reuters_word_index.json')


In [ ]:
# To view the dictionary
# my_dict

In [ ]:
# Exchange the 'key' and 'value'
my_dict_inv = {v:k for k, v in my_dict.items()}

In [ ]:
# Translate each document
i_news = 10
news = list(pd.Series(X_train[i_news]).apply(lambda x: my_dict_inv[x]))
print(' '.join(news))

1.3 Data preprocessing:

In [ ]:
# Padding: newswire lengths are uniformly matched to maxlen
# Cut away if longer than maxlen and fill with 0s if shorter than maxlen
X_train = sequence.pad_sequences(X_train, maxlen = 100)
X_test = sequence.pad_sequences(X_test, maxlen = 100)

In [ ]:
# Apply one-hot-encoding to the y variable
y = np.concatenate([y_train, y_test], axis = 0)
y = to_categorical(y, 46)
y_train = y[:n_train_size, :]
y_test = y[n_train_size:, :]

1.4 Define the model

In [ ]:
n_neurons = 100 # Neurons within each memory cell
n_input = 100 # Dimension of the embeding space

In [ ]:
# LSTM network model.
my_model = Sequential()
my_model.add(Embedding(n_words, n_input))
my_model.add(LSTM(units = n_neurons, return_sequences = False, input_shape = (None, n_input), activation='tanh'))
my_model.add(Dense(46, activation = 'softmax'))


In [ ]:
my_model.summary()

1.5 Define the optimizer and compile

In [ ]:
n_epochs = 5 # Number of epochs
batch_size = 64 # Size of each batch.
learn_rate = 0.001 # Learning rate


In [ ]:
# Optimizer and compilation.
my_optimizer = Adam(learning_rate = learn_rate)
my_model.compile(loss = 'categorical_crossentropy', optimizer = my_optimizer, metrics = ['accuracy'])

1.6 Train the model and visualize the history

In [ ]:
my_summary = my_model.fit(X_train, y_train, epochs = n_epochs, batch_size = batch_size, validation_split = 0.2, verbose=1)

In [ ]:
plt.plot(my_summary.history['accuracy'], c="b")
plt.plot(my_summary.history['val_accuracy'], c = "g")
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc ='lower right')
plt.show()

1.7 Testing

In [ ]:
ACC = my_model.evaluate(X_test, y_test, verbose = 0)
print("Test accuracy: {}".format(np.round(ACC,3)))